# Исследование базы данных приложения для чтения книг по подписке.

## Цель исследования.

Компания купила крупный сервис для чтения книг.

**Цель исследования.**

Пронализировать базу данных, в которой находятся данные о книгах, чтобы сформулировать ценностное предложение для нового продукта.

**База данных.**

База данных содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг.

## Подключение к базе данных.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {
    'user': 'praktikum_student',
    'pwd': 'Sdf4$2;d-d30pp',
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, 
    'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Обзор данных.

**Напишем функцию для выполнения SQL-запроса.**

In [4]:
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine) 

**Изучим таблицу `books`.**

In [2]:
sql='''
SELECT *
FROM books
LIMIT 5
'''

In [5]:
select(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [11]:
sql='''
SELECT count(book_id) AS cnt
FROM books
'''

In [12]:
select(sql)

,cnt
0,1000


**Таблица `books` содержит 1000 записей. В таблице представлены идентификаторы книг, авторов и издательств, а также наименования книг, количество страниц и дата издания.**

**Изучим таблицу `authors`.**

In [13]:
sql='''
SELECT *
FROM authors
LIMIT 5
'''

In [14]:
select(sql)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [15]:
sql='''
SELECT count(author_id) AS cnt
FROM authors
'''

In [16]:
select(sql)

,cnt
0,636


**Таблица `authors` содержит 636 записей об авторах книг. Таблица содержит идентификаторы и имена/псевдонимы авторов. По первым строкам таблицы можно увидеть упорядоченость по алфавиту.**

**Изучим таблицу `publishers`.**

In [17]:
sql='''
SELECT *
FROM publishers
LIMIT 5
'''

In [18]:
select(sql)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [19]:
sql='''
SELECT count(publisher_id) AS cnt
FROM publishers
'''

In [20]:
select(sql)

,cnt
0,340


**Таблица `publishers` содержит 340 записей об издательствах: идентификаторы издательств и названия. Аналогично, по первым строкам таблицы можно увидеть упорядоченность по алфавиту.**

**Изучим таблицу `ratings`.**

In [21]:
sql='''
SELECT *
FROM ratings
LIMIT 5
'''

In [22]:
select(sql)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [23]:
sql='''
SELECT count(rating_id) AS cnt
FROM ratings
'''

In [24]:
select(sql)

,cnt
0,6456


**Таблица `ratings` содержит 6456 записей. Таблица содержит информацию об идентификаторах книг и идентификаторах рейтинга, а также имя пользователя и поставленная этим пользователем оценка.**

**Изучим таблицу `reviews`.**

In [25]:
sql='''
SELECT *
FROM reviews
LIMIT 5
'''

In [26]:
select(sql)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [27]:
sql='''
SELECT count(review_id) AS cnt
FROM reviews
'''

In [28]:
select(sql)

,cnt
0,2793


**Таблица `reviews` содержит 2793 записей. Таблица содержит информацию об идентификаторах книг и обзоров, а также имя пользователя и текст обзора.**

## Исследование данных.

### Количество книг, вышедших после 1 января 2000 года.

In [30]:
sql='''
SELECT count(book_id) AS cnt
FROM books 
WHERE publication_date > '2000-01-01'
'''

In [31]:
select(sql)

,cnt
0,819


**Начиная со 2 января 2000 года было издано 819 книг.**

### Количество обзоров и средняя оценка для каждой книги.

In [36]:
sql='''
WITH average_rating AS
  (SELECT b.book_id,
          title,
          AVG(r.rating) AS average_rating
   FROM books AS b
   LEFT JOIN ratings AS r ON r.book_id = b.book_id
   GROUP BY b.book_id),
   
count_reviews AS
  (SELECT b.book_id,
          title,
          COUNT(rev.review_id) AS count_reviews
   FROM books AS b
   LEFT JOIN reviews AS rev ON rev.book_id = b.book_id
   GROUP BY b.book_id)
   
SELECT a.book_id,
       a.title, 
       c.count_reviews,
       a.average_rating
FROM average_rating AS a
JOIN count_reviews AS c ON a.book_id = c.book_id
ORDER BY a.average_rating DESC
LIMIT 5
'''

In [37]:
select(sql)

,book_id,title,count_reviews,average_rating
0,55,A Woman of Substance (Emma Harte Saga #1),2,5.0
1,57,Act of Treason (Mitch Rapp #9),2,5.0
2,17,A Dirty Job (Grim Reaper #1),4,5.0
3,20,A Fistful of Charms (The Hollows #4),2,5.0
4,62,Alas Babylon,2,5.0


**Для каждой книги мы определили количество обзоров и среднюю оценку. Отсортировав данные по убыванию можно заметить, что максимальной средней оценкой является 5. Если отсортировать данные по возрастанию, минимальная средняя оценка - 1,5.**

### Определение издательства, которое выпустили наибольшее число книг толще 50 страниц.

**Отсекая книги тоньше 50 страниц, мы исключаем из анализа брошюры.**

In [70]:

sql='''
SELECT p.publisher_id,
       p.publisher,
       COUNT(book_id) AS cnt
FROM books AS b
JOIN publishers AS p ON p.publisher_id = b.publisher_id
WHERE num_pages > 50
GROUP BY p.publisher_id,
         p.publisher
HAVING COUNT(book_id) =
  (SELECT MAX(cnt)
   FROM
     (SELECT COUNT(book_id) AS cnt
      FROM books AS b
      WHERE num_pages > 50
      GROUP BY b.publisher_id) AS tbl)
'''

In [71]:
select(sql)

,publisher_id,publisher,cnt
0,212,Penguin Books,42


**Больше всего книг (42 штуки) объемом более 50 страниц выпустило издательство Penguin Books.**

### Определение автора с самой высокой средней оценкой книг. 

**Будем учитывать только книги с 50 и более оценками.**

In [75]:
sql='''
WITH TOP AS
  (SELECT *,
          RANK() OVER (
                       ORDER BY tbl.avg_rating DESC) AS rank
   FROM
     (SELECT a.author_id,
             a.author,
             AVG(r.rating) AS avg_rating
      FROM books AS b
      LEFT JOIN authors AS a ON a.author_id = b.author_id
      LEFT JOIN ratings AS r ON r.book_id = b.book_id
      GROUP BY a.author_id,
               a.author
      HAVING (COUNT(r.rating_id) >= 50)) AS tbl)
SELECT top.author_id,
       top.author,
       top.avg_rating
FROM TOP
WHERE top.rank=1
'''

In [76]:
select(sql)

,author_id,author,avg_rating
0,130,Diana Gabaldon,4.3


**Автором, у которого средняя оценка всех его книг, которые имеют более 50 оценок, является Diana Gabaldon. Средняя оценка равна 4,3.**

### Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [78]:
sql='''
WITH users_more_48 AS
  (SELECT r.username
   FROM ratings AS r
   GROUP BY r.username
   HAVING (COUNT(r.rating_id) > 48))
SELECT AVG(tbl.cnt) AS avg_review
FROM
  (SELECT COUNT(rev.review_id) AS cnt
   FROM reviews AS rev
   WHERE rev.username in
       (SELECT *
        FROM users_more_48)
   GROUP BY rev.username) AS tbl
'''

In [79]:
select(sql)

,avg_review
0,24.0


**Пользователи, поставившие больше 48 оценок, пишут в среднем 24 обзора на книги.**

## Вывод.

**В ходе анализа базы данных были определены следующие факты:**

- в базе данных хранится информация о 1000 книг;
- 819 книг из 1000 были изданы после 1 января 2000 года;
- максимальная средняя оценка книги является 5, а минимальная средняя оценка - 1.5;
- 42 книги выпустило издательство `Penguin Books`;
- автором с самой высокой оценкой является `Diana Gabaldon`;
- пользователи, оставляющие более 48 оценок к книгам, в среднем пишут по 24 обзора на книги.